In [1]:
import os
import json
import pandas as pd

In [ ]:
result_path = 'batch_5_src_no_sports/results'
result_files = []
skip_sample_ids = {}
skip_samples = {
    5: range(40, 50), 
    10: range(10,20),
    11: range(10),
    12: range(20, 30), 
    15: range(40, 50)
}

for batch_id in range(1,16+1):
    file_name = os.path.join(result_path, f"batch_{batch_id}_annotation.json")
    result_files.append(file_name)
    if batch_id in skip_samples:
        skip_sample_ids[file_name] = skip_samples[batch_id]

final_data = {'source':[], 'summary':[], 'label':[]}
for file_path in result_files:        
    data = json.load(open(file_path))
    sample_count = 0
    for sample in data:
        sample_id = sample['sample_id']
        if file_path in skip_sample_ids and sample_id in skip_sample_ids[file_path]:
            continue
        # model_name = sample['meta_model']
        annotations = sample['annotations']
        final_data['source'].append(sample['source'])
        final_data['summary'].append(sample['summary'])
        sample_annotations = []
        occurred_annotators = set()
        for annotation in annotations:
            sample_annotations.extend(annotation['label'])   
        sample_annotations = set(sample_annotations)
        # human annotation
        if "Unwanted" in sample_annotations:
            sample_pred = "Unwanted"
        elif 'Questionable' in sample_annotations:
            sample_pred = "Questionable"
        elif "Benign" in sample_annotations:
            sample_pred = "Benign"
        else:
            sample_pred = "Consistent"
        final_data['label'].append(sample_pred)

final_data_df = pd.DataFrame.from_dict(final_data, orient='columns') 
final_data_df.to_csv('FaithBench.csv', mode='w', index=False, header=True)
        
            